In [ ]:

import pandas as pd
import random
import time
import socket

CSV_FILE = "./group68_http_input.csv" 

df = pd.read_csv(CSV_FILE)
print("Loaded CSV:")
df.head()

Loaded CSV:


,msg_id,app_protocol,src_app,src_port,dst_app,dst_port,message,timestamp
0,1,HTTP,client_browser,54321,web_server,80,GET /index.html HTTP/1.1,0.005
1,2,HTTP,web_server,80,client_browser,54321,HTTP/1.1 200 OK,0.012
2,3,HTTP,client_browser,54321,web_server,80,GET /css/styles.css HTTP/1.1,0.025
3,4,HTTP,web_server,80,client_browser,54321,HTTP/1.1 200 OK,0.035
4,5,HTTP,client_browser,54321,web_server,80,GET /js/app.js HTTP/1.1,0.045


In [ ]:
def simulate_encapsulation(row, src_ip="192.168.0.106", dst_ip="192.168.0.106"):
  
    msg = row["message"]
    
    # Application
    application_layer = {
        "protocol": row["app_protocol"],
        "payload": msg
    }
    
    # Transport– TCP 
    transport_layer = {
        "protocol": "TCP",
        "src_port": random.randint(1024, 65535),
        "dst_port": 10000,
        "seq_num": random.randint(1000, 5000),
        "ack_num": random.randint(5000, 9000),
        "flags": "PSH, ACK"
    }
    
    # Network– IP
    network_layer = {
        "protocol": "IPv4",
        "src_ip": src_ip,
        "dst_ip": dst_ip,
        "ttl": 64
    }
    
    # Data Link– Link
    link_layer = {
        "src_mac": "AA:BB:CC:DD:EE:01",
        "dst_mac": "AA:BB:CC:DD:EE:02",
        "type": "Ethernet II"
    }
    
    packet = {
        "msg_id": row["msg_id"],
        "timestamp": row["timestamp"],
        "application": application_layer,
        "transport": transport_layer,
        "network": network_layer,
        "link": link_layer
    }
    return packet

packets = [simulate_encapsulation(row) for _, row in df.iterrows()]
len(packets), packets[0]


(89,
 {'msg_id': 1,
  'timestamp': 0.005,
  'application': {'protocol': 'HTTP', 'payload': 'GET /index.html HTTP/1.1'},
  'transport': {'protocol': 'TCP',
   'src_port': 31434,
   'dst_port': 10000,
   'seq_num': 4261,
   'ack_num': 8752,
   'flags': 'PSH, ACK'},
  'network': {'protocol': 'IPv4',
   'src_ip': '192.168.0.106',
   'dst_ip': '192.168.0.106',
   'ttl': 64},
  'link': {'src_mac': 'AA:BB:CC:DD:EE:01',
   'dst_mac': 'AA:BB:CC:DD:EE:02',
   'type': 'Ethernet II'}})

In [ ]:
# DataFrame 
records = []
for p in packets:
    rec = {
        "msg_id": p["msg_id"],
        "timestamp": p["timestamp"],
        "app_protocol": p["application"]["protocol"],
        "app_payload": p["application"]["payload"],
        "tcp_src_port": p["transport"]["src_port"],
        "tcp_dst_port": p["transport"]["dst_port"],
        "tcp_flags": p["transport"]["flags"],
        "ip_src": p["network"]["src_ip"],
        "ip_dst": p["network"]["dst_ip"],
        "link_src_mac": p["link"]["src_mac"],
        "link_dst_mac": p["link"]["dst_mac"],
    }
    records.append(rec)

packets_df = pd.DataFrame.from_records(records)
packets_df


,msg_id,timestamp,app_protocol,app_payload,tcp_src_port,tcp_dst_port,tcp_flags,ip_src,ip_dst,link_src_mac,link_dst_mac
0,1,0.005,HTTP,GET /index.html HTTP/1.1,31434,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
1,2,0.012,HTTP,HTTP/1.1 200 OK,10433,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
2,3,0.025,HTTP,GET /css/styles.css HTTP/1.1,21945,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
3,4,0.035,HTTP,HTTP/1.1 200 OK,23239,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
4,5,0.045,HTTP,GET /js/app.js HTTP/1.1,47965,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
...,...,...,...,...,...,...,...,...,...,...,...
84,85,0.845,HTTP,GET /ads/banner.jpg HTTP/1.1,2537,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
85,86,0.855,HTTP,HTTP/1.1 200 OK,49831,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
86,87,0.865,HTTP,POST /api/track_click HTTP/1.1,1358,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02
87,88,0.875,HTTP,HTTP/1.1 200 OK,44653,10000,"PSH, ACK",192.168.0.106,192.168.0.106,AA:BB:CC:DD:EE:01,AA:BB:CC:DD:EE:02


In [15]:
HOST = "192.168.0.106"  
PORT = 10000

def send_http_traffic(df, host=HOST, port=PORT, delay=0.3):
  
    for _, row in df.iterrows():
        message = row["message"]
        
        if "HTTP/1.1" in message and "Host:" not in message:
            http_request = (
                f"{message}\r\n"
                f"Host: {host}\r\n"
                f"Connection: close\r\n"
                f"\r\n"
            )
        else:
            http_request = message + "\r\n\r\n"
        
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((host, port))
            s.sendall(http_request.encode("utf-8"))
            _ = s.recv(1024)
        except Exception as e:
            print(f"Error sending message {row['msg_id']}: {e}")
        finally:
            s.close()
        
        time.sleep(delay)

print("Ready. Run send_http_traffic(df) after capturing Wireshark.")


Ready. Run send_http_traffic(df) after capturing Wireshark.


## הוראות לכידת תעבורה ב-Wireshark

1. פתחו את Wireshark.
2. בחרו בממשק הרשת שבו אתם משתמשים.
3. הגדירו פילטר לדוגמה:
   ```wireshark
   tcp.port == 10000

In [16]:
send_http_traffic(df)